In [134]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

In [135]:
v1=json.load(open('../../mc1_vis_system/public/MC1.json'))
nodes_df=pd.DataFrame(v1['nodes'])
links_df=pd.DataFrame(v1['links'])
merged_info=pd.read_csv('./merged_info.csv') #用原始的，算连边数量最多的点
merged_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11069 entries, 0 to 11068
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      11069 non-null  int64  
 1   link_type       11069 non-null  object 
 2   weight          11068 non-null  float64
 3   dataset_x       11069 non-null  object 
 4   source          11069 non-null  object 
 5   target          11069 non-null  object 
 6   key             11069 non-null  int64  
 7   source_type     11069 non-null  object 
 8   source_country  3003 non-null   object 
 9   target_type     11069 non-null  object 
 10  target_country  3547 non-null   object 
dtypes: float64(1), int64(2), object(8)
memory usage: 951.4+ KB


In [136]:
#{name:,in_num:,out_num:,
# owner_num:,partner_num:,member_num:,fam_num:,
# undefined_num:,company:,organization:,person:,location:,movement:,event:,political_organization:,vessel:,
# top3neighbor:[]}

In [137]:
statistics={}
out_num={}
in_num={}
# 出边统计
out_group=list(merged_info.groupby('source'))
for i in range(len(out_group)):
    source,df=out_group[i]
    out_num[source]=len(df)
# 入边统计 
in_group=list(merged_info.groupby('target'))
for i in range(len(in_group)):
    target,df=in_group[i]
    in_num[target]=len(df)
    


In [138]:
# out_num['Oceanfront Oasis Inc Carriers'],in_num['0']

In [139]:
# 直连边类型统计
neighbor_link_type={}
for i in range(len(out_group)):
    source,df=out_group[i]
    type_cnt=df['link_type'].value_counts()
    neighbor_link_type[source]=type_cnt.to_dict()
for i in range(len(in_group)):
    target,df=in_group[i]
    type_cnt=df['link_type'].value_counts().to_dict()
    if target in neighbor_link_type:
        tmp=Counter(neighbor_link_type[target])
        neighbor_link_type[target]=dict(tmp+Counter(type_cnt))
    else: 
        neighbor_link_type[target]=type_cnt

neighbor_link_type['0']

{'family_relationship': 6, 'ownership': 2, 'partnership': 5}

In [140]:
# 邻居节点统计
neighbor_node_type={}
for i in range(len(out_group)):
    source,df=out_group[i]
    type_cnt=df['target_type'].value_counts()
    neighbor_node_type[source]=type_cnt.to_dict()
for i in range(len(in_group)):
    target,df=in_group[i]
    type_cnt=df['source_type'].value_counts().to_dict()
    if target in neighbor_node_type:
        tmp=Counter(neighbor_node_type[target])
        neighbor_node_type[target]=dict(tmp+Counter(type_cnt))
    else: 
        neighbor_node_type[target]=type_cnt
neighbor_node_type['0']

{'organization': 7, 'person': 2, 'unknown': 3, 'location': 1}

In [141]:
# 连边最多的top3
top3={}
for i in range(len(out_group)):
    source,df=out_group[i]
    top3[source]=(df['target'].value_counts().to_dict())
for i in range(len(in_group)):
    target,df=in_group[i]
    src_cnt=(df['source'].value_counts().to_dict())
    if target in top3:
        tmp=Counter(top3[target])
        top3[target]=dict(tmp+Counter(src_cnt))
    else:
        top3[target]=src_cnt
for key in top3:
    top3[key]=sorted(top3[key].items(), key=lambda x: x[1], reverse=True)[:3]

top3['8327']

[('435054320', 2), ('7505050', 2), ('185106', 1)]

In [142]:
top3['979893388']

[('Mar del Oeste Pic', 6),
 ('903311212', 6),
 ('SeaSpray Wave SRL Solutions', 6)]

### 处理unity

In [143]:
# 打开json
with open("../../data/unities.json", "r") as file:
    unity_data = json.load(file)

In [144]:
len(unity_data)

4

In [145]:
for i in unity_data:
    cnt=0
    name=i['name'].split("_")[0]
    if name=="person":
        name="family"
    name=name.capitalize()
    new_unity=[]
    for j in i['unity']:
        # print(j)
        record_dict={}
        record_dict['name']=name+' Group '+str(cnt)
        cnt+=1
        record_dict['members']=j
        record_dict['size']=len(j)
        new_unity.append(record_dict)
        # i['unity'].append(record_dict)
    i['unity']=new_unity

In [146]:
# 写成新的json
with open("../../data/new_unities.json", "w") as file:
    json.dump(unity_data, file)

In [147]:
nodes_df['type'].fillna("null",inplace=True)
nodes_df['country'].fillna("null",inplace=True)

### 写json

In [148]:
all_nodes=nodes_df['id'].unique()
# node_dict_keys=["name","in_num","out_num",
# "owner_num","partner_num","member_num","fam_num",
# "undefined","company","organization","person","location","movement","event","political_organization","vessel",
# "top3neighbor"]
link_keys=["ownership","partnership","membership","family_relationship"]
node_keys=["null","company","organization","person","location","movement","event","political_organization","vessel"]
for node in all_nodes:
    node_dict={}
    # node_dict[node]={}
    tmp_node_df=nodes_df[nodes_df['id']==node]
    node_dict['type']=tmp_node_df['type'].values[0]
    node_dict['country']=tmp_node_df['country'].values[0] 
    node_dict['in_num']=in_num[node] if node in in_num else 0
    node_dict['out_num']=out_num[node] if node in out_num else 0
    node_link=neighbor_link_type[node]
    node_node=neighbor_node_type[node]
    for key in link_keys:
        # print(key)
        node_dict[key]=node_link[key] if key in node_link else 0
    
    for key in node_keys:
        node_dict[key]=node_node[key] if key in node_node else 0
    node_dict["null"]=node_node["null"] if "null" in node_node else 0
    node_dict['top3neighbor']=top3[node]
    statistics[node]=(node_dict)
    

#### unity信息放进dict

In [149]:
for i in unity_data:
    for j in i['unity']:
        name=j['name']
        for mem in j['members']:
            # print(mem)
            if mem=="OceanicOrigin Foods Co and Sons":
                mem="OceanicOrigin Foods Co and Son's"
            if mem in statistics:
                # print(mem)
                statistics[mem]['unity_name']=name
                statistics[mem]['unity_size']=j['size']
for i in statistics:
    if 'unity_name' not in statistics[i]:
        statistics[i]['unity_name']="null"
        statistics[i]['unity_size']="null"
            
        

##### 弱连通分量里的船

In [150]:
# 读取json
with open("../../../技术储备/Tarjan/weakly connected component.json", "r") as file:
    wcc = json.load(file)

In [151]:
all_ves=nodes_df[nodes_df['type']=='vessel']['id'].unique()
for i in wcc:
    components=i['component']
    ves_in_comp=[i for i in components if i in all_ves]
    if len(ves_in_comp)!=0:
        # 该wcc里有vessel
        for j in components:
            statistics[j]['vessels_in_same_wcc']=ves_in_comp
    else:
        for j in components:
            statistics[j]['vessels_in_same_wcc']="null"


In [152]:
a=[1,2]
b=a.remove(2)
b

##### 同一弱连通分量里出入度之和最大的top3
（也有可能同一弱连通分量里没有超过4个点，那就展示所有点）

In [153]:
merged_info=pd.read_csv('./merged_info.csv')
for i in wcc:
    components=i['component']
    sum_degrees={}
    for j in components:
        sum_degrees[j]=len(merged_info[(merged_info['source']==j) | (merged_info['target']==j)])
    sum_degrees_top=sorted(sum_degrees.items(), key=lambda x: x[1], reverse=True)
    top3=sum_degrees_top[:3]
    for j in components:
        tmp=top3.copy()
        statistics[j]['sum_degrees_top3']=tmp
        top3node=[i[0] for i in top3]
        if j in top3node:
            statistics[j]['sum_degrees_top3'].remove((j,sum_degrees[j]))
            # print(j, statistics[j]['sum_degrees_top3'])
            if len(sum_degrees_top)>=4:
                statistics[j]['sum_degrees_top3'].append(sum_degrees_top[3])
            


In [154]:
statistics['Ocean Fisheries Llc']

{'type': 'null',
 'country': 'null',
 'in_num': 193,
 'out_num': 207,
 'ownership': 96,
 'partnership': 79,
 'membership': 168,
 'family_relationship': 57,
 'null': 0,
 'company': 62,
 'organization': 112,
 'person': 80,
 'location': 37,
 'movement': 2,
 'event': 7,
 'political_organization': 9,
 'vessel': 12,
 'top3neighbor': [('SeaSpray Wave SRL Solutions', 40),
  ('png xi  Line', 28),
  ('903311212', 24)],
 'unity_name': 'null',
 'unity_size': 'null',
 'vessels_in_same_wcc': ['FV',
  'Motor Vessel',
  'boatâ\x80\x99s',
  'Cutter',
  'AirPods',
  'Water',
  'USCGC',
  'ARA Contraalmirante Cordero',
  'wheeler',
  'HMS',
  'salmon',
  'HMS Medway',
  'USS',
  'Ocean Patrol',
  'river',
  'Manned',
  'Gulf',
  'FV Thunder',
  'dup621',
  'PC',
  'bulker',
  'Wartsila',
  'Coast',
  'Oceans',
  'Rapido beam',
  'Hang',
  'Vehicle Land',
  'FV Snake',
  '629',
  'Deepwater Horizon',
  'Craftâ\x80\x93Law',
  'reefers',
  'water',
  'Coast Guard Cutter Edgar Culbertson',
  'Viking fishing'

##### 用强连通的更新强连通的来源

In [155]:
# 读取json
with open("../../../技术储备/Tarjan/strong connected component.json", "r") as file:
    scc = json.load(file)

In [156]:
all_ves=nodes_df[nodes_df['type']=='vessel']['id'].unique()
for i in scc:
    components=i['component']
    ves_in_comp=[i for i in components if i in all_ves]
    if len(ves_in_comp)!=0:
        # 该scc里有vessel
        for j in components:
            statistics[j]['vessels_in_same_scc']=ves_in_comp
            statistics[j]['vessels_in_same_wcc']="null"
    

In [157]:
for a in statistics:
    if "vessels_in_same_scc" not in statistics[a]:
        statistics[a]['vessels_in_same_scc']="null"

In [158]:
statistics['979893388']

{'type': 'organization',
 'country': 'null',
 'in_num': 124,
 'out_num': 81,
 'ownership': 22,
 'partnership': 45,
 'membership': 68,
 'family_relationship': 70,
 'null': 0,
 'company': 16,
 'organization': 64,
 'person': 51,
 'location': 17,
 'movement': 3,
 'event': 2,
 'political_organization': 10,
 'vessel': 1,
 'top3neighbor': [('Mar del Oeste Pic', 6),
  ('903311212', 6),
  ('SeaSpray Wave SRL Solutions', 6)],
 'unity_name': 'Organization Group 1',
 'unity_size': 134,
 'vessels_in_same_wcc': 'null',
 'sum_degrees_top3': [('Ocean Fisheries Llc', 400),
  ('png xi  Line', 399),
  ('SeaSpray Wave SRL Solutions', 391)],
 'vessels_in_same_scc': ['Talleyâ\x80\x99s',
  'ocean',
  '629',
  'Deepwater Horizon',
  'Plastic',
  '19',
  'dup77',
  'water',
  'Ships',
  'salmon',
  'USS',
  'trucking',
  'Gulf',
  'Oliver',
  'bottom',
  'PC',
  'Vessels',
  'HMS Montrose',
  'dup626']}

In [159]:
for b in statistics:
    statistics[b]['scc_sum_degrees_top3']="null"

In [160]:
# merged_info=pd.read_csv('./merged_info.csv')
for i in scc:
    components=i['component']
    sum_degrees={}
    for j in components:
        sum_degrees[j]=len(merged_info[(merged_info['source']==j) | (merged_info['target']==j)])
    sum_degrees_top=sorted(sum_degrees.items(), key=lambda x: x[1], reverse=True)
    top3=sum_degrees_top[:3]
    for j in components:
        tmp=top3.copy()
        statistics[j]['scc_sum_degrees_top3']=tmp
        statistics[j]['sum_degrees_top3']="null"
        top3node=[i[0] for i in top3]
        if j in top3node:
            statistics[j]['scc_sum_degrees_top3'].remove((j,sum_degrees[j]))
            # print(j, statistics[j]['sum_degrees_top3'])
            if len(sum_degrees_top)>=4:
                statistics[j]['scc_sum_degrees_top3'].append(sum_degrees_top[3])
            


In [161]:
statistics["Dawn Robinson"]["scc_sum_degrees_top3"]

'null'

In [164]:
statistics["979893388"]

{'type': 'organization',
 'country': 'null',
 'in_num': 124,
 'out_num': 81,
 'ownership': 22,
 'partnership': 45,
 'membership': 68,
 'family_relationship': 70,
 'null': 0,
 'company': 16,
 'organization': 64,
 'person': 51,
 'location': 17,
 'movement': 3,
 'event': 2,
 'political_organization': 10,
 'vessel': 1,
 'top3neighbor': [('Mar del Oeste Pic', 6),
  ('903311212', 6),
  ('SeaSpray Wave SRL Solutions', 6)],
 'unity_name': 'Organization Group 1',
 'unity_size': 134,
 'vessels_in_same_wcc': 'null',
 'sum_degrees_top3': 'null',
 'vessels_in_same_scc': ['Talleyâ\x80\x99s',
  'ocean',
  '629',
  'Deepwater Horizon',
  'Plastic',
  '19',
  'dup77',
  'water',
  'Ships',
  'salmon',
  'USS',
  'trucking',
  'Gulf',
  'Oliver',
  'bottom',
  'PC',
  'Vessels',
  'HMS Montrose',
  'dup626'],
 'scc_sum_degrees_top3': [('Ocean Fisheries Llc', 400),
  ('png xi  Line', 399),
  ('SeaSpray Wave SRL Solutions', 391)]}

In [165]:
with open("../../data/card_statistics.json",'w',encoding='utf-8') as f:
    f.write(json.dumps(statistics))
    